In [1]:
import xarray as xr 

import sys
sys.path.append('/home/links/ct715/eddy_feedback')
import functions.eddy_feedback as ef 

In [2]:
ds = xr.open_mfdataset('/home/links/ct715/eddy_feedback/daily_datasets/jra55_djf_ep.nc')
ds

<xarray.Dataset>
Dimensions:  (time: 5325, lon: 144, lat: 73, level: 37)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01T09:00:00 ... 2016-12-31T09:00:00
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    v        (time, level, lat, lon) float32 dask.array<chunksize=(1, 37, 73, 144), meta=np.ndarray>
    t        (time, level, lat, lon) float32 dask.array<chunksize=(1, 37, 73, 144), meta=np.ndarray>
    omega    (time, level, lat, lon) float32 dask.array<chunksize=(1, 37, 73, 144), meta=np.ndarray>
    u        (time, level, lat, lon) float32 dask.array<chunksize=(1, 37, 73, 144), meta=np.ndarray>
    ep1      (time, level, lat) float64 dask.array<chunksize=(1, 37, 73), meta=np.ndarray>
    ep2      (time, level, lat) float64 dask.array<chunksize=(1, 37, 73), meta=np.ndarray>
    div1     (time, level, lat) float64 dask.array<chunksize=(1, 37, 73), meta=np.ndarray>
    div2     (time, level, lat) float64 dask.array<chunksize=(1, 37, 73), meta=np.ndarray>
    ubar     (level, lat) float32 dask.array<chunksize=(37, 73), meta=np.ndarray>

In [15]:
def calculate_ef_parameter(ds, primitive=True, hemisphere='NH', top_atmos=10., reanalysis=True,
                           whole_area=True, which_level=None):

    """ 
    Input: Xarray dataset

    Output: Xarray dataArray of Eddy Feedback Parameter (EFP)
    """

    ## CONDITIONS
    
    # ensure variables are named correctly
    if 'lat' and 'lon' and 'level' and 'u' and 'v' and 't' not in ds:
        ds = ef.find_rename_variables(ds)  
    
    # Check to see if EP fluxes are in DataSet
    if not 'ep1' in ds:
        ds = ef.calculate_epfluxes_ubar(ds, primitive=primitive)


    # Reduce dataset to 200-600hPa
    ds = ds.where( ds.level >= 200., drop=True ) 
    ds = ds.where( ds.level <= 600., drop=True ) 

    # choose hemisphere
    if hemisphere == 'SH':
        # set southern hermisphere
        ds = ds.where( ds.lat <= 0, drop=True )
    else:
        # set northern hemisphere
        ds = ds.where( ds.lat >= 0, drop=True )
    
    # cut off stratosphere
    ds = ds.where( ds.level >= top_atmos, drop=True )

    #-------------------------------------------------------------------

    ## SET UP TIME
    
    # remove unwanted variables
    vars = ['u', 'div1', 'div2']
    ds = ds[vars]
    
    # set variables and save them
    ubar = ds.u.mean(('lon'))
    div1 = ds.div1
    div2 = ds.div2
    
    if reanalysis == True:
        # separate time into annual means
        # and use .load() to force the calculation now
        ubar = ubar.groupby('time.year').mean('time').load()
        div1 = div1.groupby('time.year').mean('time').load()
        div2 = div2.groupby('time.year').mean('time').load()
    else:
        # separate time into annual means
        ubar = ubar.load()
        div1 = div1.load()
        div2 = div2.load()


    R = ef.correlation_array(ubar, div1)

    # # choose between whole area or individual levels
    # if whole_area == True:
    #     # average over 200-600hPa
    #     R = R.mean(('lon'))
    # else:
    #     # take a slice at particular level
    #     R = R.sel( ds.level == which_level)


    return R 

In [16]:
R1 = calculate_ef_parameter(ds)
R1

array([[-0.06723912, -0.19346344, -0.0478184 ,  0.12805259,  0.23323409,
         0.22654035,  0.22432979,  0.2628565 ,  0.33872312,  0.48685046,
         0.64325903,  0.71120758,  0.72529781,  0.7044347 ,  0.64271207,
         0.57607459,  0.52971558,  0.45967913,  0.37879982,  0.36780853,
         0.47249972,  0.50600856,  0.53215033,  0.61235938,  0.53613184,
         0.33670234,  0.20139622,  0.21069995,  0.29063102,  0.34446026,
         0.37952688,  0.30889575,  0.11617941, -0.06918076, -0.21122391,
        -0.14459719,  0.15801622],
       [-0.01382241, -0.22487746, -0.11934022,  0.08485404,  0.19535177,
         0.18422118,  0.18272174,  0.23686751,  0.32939015,  0.47772315,
         0.65079458,  0.70533005,  0.69511782,  0.69224144,  0.6699068 ,
         0.63562216,  0.59353747,  0.49148118,  0.40564789,  0.42555192,
         0.53247644,  0.56373822,  0.52386372,  0.5760663 ,  0.5603744 ,
         0.50029865,  0.41451785,  0.26567019,  0.20690147,  0.20028539,
         0.28159

In [17]:
R1.shape

(10, 37)

In [18]:
def calculate_ef_parameter(ds, primitive=True, hemisphere='NH', top_atmos=10., reanalysis=True,
                           whole_area=True, which_level=None):

    """ 
    Input: Xarray dataset

    Output: Xarray dataArray of Eddy Feedback Parameter (EFP)
    """

    ## CONDITIONS
    
    # ensure variables are named correctly
    if 'lat' and 'lon' and 'level' and 'u' and 'v' and 't' not in ds:
        ds = ef.find_rename_variables(ds)  
    
    # Check to see if EP fluxes are in DataSet
    if not 'ep1' in ds:
        ds = ef.calculate_epfluxes_ubar(ds, primitive=primitive)


    # Reduce dataset to 200-600hPa
    ds = ds.where( ds.level >= 200., drop=True ) 
    ds = ds.where( ds.level <= 600., drop=True ) 

    # choose hemisphere
    if hemisphere == 'SH':
        # set southern hermisphere
        ds = ds.where( ds.lat <= 0, drop=True )
    else:
        # set northern hemisphere
        ds = ds.where( ds.lat >= 0, drop=True )
    
    # cut off stratosphere
    ds = ds.where( ds.level >= top_atmos, drop=True )

    #-------------------------------------------------------------------

    ## SET UP TIME
    
    # remove unwanted variables
    vars = ['u', 'div1', 'div2']
    ds = ds[vars]
    
    # set variables and save them
    ubar = ds.u.mean(('lon'))
    div1 = ds.div1
    div2 = ds.div2
    
    if reanalysis == True:
        # separate time into annual means
        # and use .load() to force the calculation now
        ubar = ubar.groupby('time.year').mean('time').load()
        div1 = div1.groupby('time.year').mean('time').load()
        div2 = div2.groupby('time.year').mean('time').load()
    else:
        # separate time into annual means
        ubar = ubar.load()
        div1 = div1.load()
        div2 = div2.load()


    R = xr.corr(ubar, div1, dim='year')

    # # choose between whole area or individual levels
    # if whole_area == True:
    #     # average over 200-600hPa
    #     R = R.mean(('lon'))
    # else:
    #     # take a slice at particular level
    #     R = R.sel( ds.level == which_level)


    return R 

In [19]:
R = calculate_ef_parameter(ds)
R

<xarray.DataArray (level: 10, lat: 37)>
array([[-0.06723913, -0.19346345, -0.04781839,  0.12805259,  0.23323409,
         0.22654036,  0.22432978,  0.26285647,  0.33872312,  0.48685046,
         0.64325905,  0.71120759,  0.72529785,  0.70443471,  0.64271214,
         0.57607462,  0.52971561,  0.45967913,  0.37879983,  0.36780851,
         0.47249972,  0.50600858,  0.53215036,  0.61235937,  0.5361319 ,
         0.33670233,  0.20139622,  0.21069993,  0.29063099,  0.34446023,
         0.37952686,  0.30889574,  0.1161794 , -0.06918077, -0.21122391,
        -0.14459717,  0.15801622],
       [-0.0138224 , -0.22487746, -0.11934021,  0.08485405,  0.19535177,
         0.18422118,  0.18272173,  0.23686749,  0.32939012,  0.47772313,
         0.65079452,  0.70532997,  0.69511784,  0.69224149,  0.66990676,
         0.63562208,  0.59353748,  0.49148115,  0.4056479 ,  0.42555192,
         0.53247647,  0.56373821,  0.52386375,  0.57606627,  0.56037442,
         0.50029865,  0.41451786,  0.26567018,  0.20690147,  0.20028537,
         0.28159079,  0.28880374,  0.14920548, -0.03552754, -0.20322745,
        -0.14960552,  0.15862892],
       [ 0.40526423, -0.23192342, -0.15182911,  0.07346337,  0.1765399 ,
         0.15729744,  0.1546217 ,  0.21947516,  0.32652261,  0.46558985,
         0.6390944 ,  0.68688386,  0.67240061,  0.68100556,  0.69114362,
         0.67920105,  0.63384411,  0.53579243,  0.44824332,  0.46526041,
...
         0.42041583,  0.45393878,  0.64945883,  0.65243856,  0.56028142,
         0.49294627,  0.42852021,  0.37166353,  0.23711893, -0.05980539,
        -0.2778317 , -0.32825577, -0.34965135, -0.27214811, -0.18486013,
        -0.25375809, -0.2226536 ],
       [ 0.07104706, -0.14835268, -0.26192474, -0.03537613,  0.09802386,
         0.07165988, -0.05081372, -0.10532013,  0.09059087,  0.34338604,
         0.49295999,  0.61390179,  0.6218825 ,  0.46914112,  0.36849657,
         0.35266376,  0.43831567,  0.46043304,  0.44292579,  0.44237921,
         0.29943021,  0.32217135,  0.62431037,  0.61597103,  0.58094653,
         0.54447392,  0.42487219,  0.35478081,  0.24682939,  0.02805799,
        -0.1510991 , -0.30890349, -0.38155518, -0.31788391, -0.30436266,
        -0.12977743,  0.00811221],
       [-0.0124092 , -0.22281437, -0.27467934, -0.03958142,  0.15169983,
         0.09414668, -0.11214533, -0.22022152,  0.0019298 ,  0.32615878,
         0.47861291,  0.62773248,  0.62451696,  0.43512109,  0.31582992,
         0.23207258,  0.30350414,  0.35780178,  0.3034297 ,  0.28609205,
         0.14087459,  0.24642067,  0.63271509,  0.60477378,  0.57293365,
         0.56288987,  0.42236867,  0.40020358,  0.30828245,  0.0230929 ,
        -0.16354806, -0.33345257, -0.44141115, -0.37440523, -0.28455374,
        -0.01212327,  0.11644778]])
Coordinates:
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... 10.0 7.5 5.0 2.5 0.0
  * level    (level) float64 200.0 225.0 250.0 300.0 ... 450.0 500.0 550.0 600.0

In [25]:
(R.values - R1).shape

(10, 37)

In [21]:
R1.shape

(10, 37)

In [22]:
R.shape

(10, 37)